In [74]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import subprocess
import time
from tqdm import tqdm

In [62]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

In [73]:
# TEST RUN
def extract_metrics(url):
    #url = 'https://www.kaggle.com/code/m1nty03/rice-type-classification'
    name = url.split("www.kaggle.com/code/")
    name = name[-1].replace("/", "--")
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'rendered-kernel-content'))
    )
    
    iframe = driver.find_element(By.ID, 'rendered-kernel-content')
    iframe_src = iframe.get_attribute('src')
    response = requests.get(iframe_src)
    soup = bs(response.text, 'html.parser')
    
    pattern1 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)'
    pattern2 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)(?:.*? - val_accuracy:\s*(\d+\.\d+)\s*- val_loss:\s*(\d+\.\d+))?'
    matches = re.findall(pattern2, soup.text, re.DOTALL)
    
    structured_data = []
    for match in matches:
        epoch = int(match[0])
        acc = float(match[1])
        loss = float(match[2])
        val_acc = float(match[3]) if match[3] else None
        val_loss = float(match[4]) if match[4] else None
        structured_data.append((epoch, acc, loss, val_acc, val_loss))
    
    # Create a DataFrame for better readability
    df = pd.DataFrame(structured_data, columns=['Epoch', 'Accuracy', 'Loss', 'Val_Accuracy', 'Val_Loss'])
    if len(df) > 0:
        print(f"found {len(df)} metrics for {name}")
        df.to_csv(name + ".csv", index=False)
        return df
    else:
        return

df = extract_metrics("https://www.kaggle.com/code/adhamtarek147/cifar-10")


'ref'

In [84]:
refs = pd.read_csv("output.txt")
for ref in tqdm(list(refs["ref"])[49:]):
    url = "https://www.kaggle.com/code/" + ref
    #print(f"processing url {url}")
    try:
        extract_metrics(url)
    except Exception as e:
        print(url, e)

  0%|▏                                                                              | 2/1021 [00:14<2:12:06,  7.78s/it]

https://www.kaggle.com/code/ref Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79796B095+29557]
	(No symbol) [0x00007FF7978DFA50]
	(No symbol) [0x00007FF79779B56A]
	(No symbol) [0x00007FF7977EF695]
	(No symbol) [0x00007FF7977EF8EC]
	(No symbol) [0x00007FF79783B777]
	(No symbol) [0x00007FF7978171CF]
	(No symbol) [0x00007FF79783851C]
	(No symbol) [0x00007FF797816F33]
	(No symbol) [0x00007FF7977E116F]
	(No symbol) [0x00007FF7977E22D1]
	GetHandleVerifier [0x00007FF797C9C96D+3378253]
	GetHandleVerifier [0x00007FF797CE8497+3688311]
	GetHandleVerifier [0x00007FF797CDD1CB+3642539]
	GetHandleVerifier [0x00007FF797A2A6B6+813462]
	(No symbol) [0x00007FF7978EAB5F]
	(No symbol) [0x00007FF7978E6B74]
	(No symbol) [0x00007FF7978E6D10]
	(No symbol) [0x00007FF7978D5C1F]
	BaseThreadInitThunk [0x00007FF9018A7374+20]
	RtlUserThreadStart [0x00007FF90363CC91+33]



  3%|██▏                                                                             | 28/1021 [01:34<50:28,  3.05s/it]

found 5 metrics for moridata--nlp-ham-spam-sms


  5%|███▌                                                                            | 46/1021 [02:29<47:59,  2.95s/it]

...

found 10 metrics for furkan09--customer-churn-prediction


  7%|█████▍                                                                          | 70/1021 [03:55<59:34,  3.76s/it]

found 14 metrics for muhammedtariq99--hand-sign-multi-class-classification-cnn-97


  9%|██████▉                                                                         | 88/1021 [04:57<52:37,  3.38s/it]


KeyboardInterrupt: 

In [48]:
driver.quit()